In [1]:
import pandas as pd
import numpy as np
import torch
import time
import random
import os
from torchtext import data
from torchtext.vocab import Vectors
from torch.nn import init
from tqdm import tqdm
tokenize = lambda x: x.split()
#data.Field:定义样本的处理操作
TEXT = data.Field(sequential = True, tokenize = tokenize, lower = True, fix_length = 200)
LABEL = data.Field(sequential = False, use_vocab = False)

In [2]:
#将原始的corpus转换成data.Example实例(主要为data.Example.fromlist方法)
def get_dataset(csv_data, text_field, label_field, test=False):
    fields = [('id', None), ('comment_text', text_field), ('toxic', label_field)]
    examples = []
    if test:
        #如果为测试集，则不加载label
        for text in tqdm(csv_data['comment_text']):
            examples.append(data.Example.fromlist([None, text, None], fields))
    else:
        for text, label in tqdm(zip(csv_data['comment_text'], csv_data['toxic'])):
            examples.append(data.Example.fromlist([None, text, label], fields))
    return examples, fields

In [16]:
train_data = pd.read_csv(r'C:\Users\lenovo\Desktop\Josie\自学\TorchText\practical-torchtext-master\practical-torchtext-master\data\train.csv', engine='python')
test_data = pd.read_csv(r'C:\Users\lenovo\Desktop\Josie\自学\TorchText\practical-torchtext-master\practical-torchtext-master\data\test.csv', engine='python')
valid_data = pd.read_csv(r'C:\Users\lenovo\Desktop\Josie\自学\TorchText\practical-torchtext-master\practical-torchtext-master\data\valid.csv', engine='python')
#得到构建Dataset所需的examples和fields
train_examples, train_fields = get_dataset(train_data, TEXT, LABEL)
valid_examples, valid_fields = get_dataset(valid_data, TEXT, LABEL)
test_examples, test_fields = get_dataset(test_data, TEXT, None, test=True)
#使用torchtext.data.Dataset来构建数据集
train = data.Dataset(train_examples, train_fields)
valid = data.Dataset(valid_examples, valid_fields)
test = data.Dataset(test_examples, test_fields)

0it [00:00, ?it/s]

[0 1]
[0 1]
[0 1]


3it [00:00, 27.66it/s]

[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]


14it [00:00, 35.66it/s]

...

25it [00:00, 87.79it/s]
0it [00:00, ?it/s]

[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]


19it [00:00, 182.94it/s]

[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]


25it [00:00, 169.14it/s]
100%|████████████████████████████████████████████████████████████████████████████████| 33/33 [00:00<00:00, 8261.43it/s]


In [5]:
#对比于旧代码，旧代码在这里报错，可能是不能调用GLOVE
#旧代码
# vectors = Vectors(name='myvector/glove/glove.6B.200d.txt', cache=cache)
# TEXT.build_vocab(train, vectors=vectors)
from torchtext.vocab import GloVe, Vectors
from torchtext import data
TEXT.build_vocab(train)
weight_matrix = TEXT.vocab.vectors

In [6]:
from torchtext.data import Iterator, BucketIterator
train_iter, val_iter = BucketIterator.splits(
        (train, valid), # 构建数据集所需的数据集
        batch_sizes=(8, 8),
        # 如果使用gpu，此处将-1更换为GPU的编号
        device=-1,
        # the BucketIterator needs to be told what function it should use to group the data.
        sort_key=lambda x: len(x.comment_text),
        sort_within_batch=False,
        repeat=False
)
test_iter = Iterator(test, batch_size=8, device=-1, sort=False, sort_within_batch=False, repeat=False)

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


In [11]:
for idx, batch in enumerate(train_iter):
    text, label = batch.comment_text, batch.toxic
    print(text.shape, label.shape)

torch.Size([200, 8]) torch.Size([8])
torch.Size([200, 8]) torch.Size([8])
torch.Size([200, 8]) torch.Size([8])
torch.Size([200, 1]) torch.Size([1])


In [59]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
weight_matrix = TEXT.vocab.vectors

class LSTM(nn.Module):

    def __init__(self):
        super(LSTM, self).__init__()
        self.word_embeddings = nn.Embedding(len(TEXT.vocab), 300)  # embedding之后的shape: torch.Size([200, 8, 300])
        # 若使用预训练的词向量，需在此处指定预训练的权重
        # embedding.weight.data.copy_(weight_matrix)
        self.lstm = nn.LSTM(input_size=300, hidden_size=128, num_layers=1)  # torch.Size([200, 8, 128])
        self.decoder = nn.Linear(128, 2)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        print(embeds.shape)
        lstm_out = self.lstm(embeds)[0]  # lstm_out:200x8x128
        print(lstm_out.shape)
        # 取最后一个时间步
        final = lstm_out[-1]  # 8*128
        y = self.decoder(final)  # 8*2 
        return y

def main():
    model = LSTM()
    model.train()
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.01)
    loss_funtion = F.cross_entropy
    for epoch, batch in enumerate(val_iter):
        optimizer.zero_grad()
        start = time.time()
        predicted = model(batch.comment_text)
        loss = loss_funtion(predicted, batch.toxic)
        loss.backward()
        optimizer.step()
        print(loss)
   
if __name__ == '__main__':
    main()

torch.Size([200, 8, 300])
torch.Size([200, 8, 128])
tensor(0.8237, grad_fn=<NllLossBackward>)
torch.Size([200, 8, 300])
torch.Size([200, 8, 128])
tensor(0.5778, grad_fn=<NllLossBackward>)
torch.Size([200, 8, 300])
torch.Size([200, 8, 128])
tensor(0.5465, grad_fn=<NllLossBackward>)
torch.Size([200, 1, 300])
torch.Size([200, 1, 128])
tensor(0.6153, grad_fn=<NllLossBackward>)


In [106]:
#------我是昏割线------#

In [47]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import utils as nn_utils

batch_size = 2
max_length = 3
hidden_size = 2
n_layers = 1
tensor_in = torch.FloatTensor([[1,2,3], [1,0,0]]).resize_(2,3,1)
tensor_in = Variable(tensor_in)
seq_lengths = [3,1]
pack = nn_utils.rnn.pack_padded_sequence(tensor_in, seq_lengths, batch_first = True)
print(tensor_in.size())
print('packed:', pack)

torch.Size([2, 3, 1])
packed: PackedSequence(data=tensor([[1.],
        [1.],
        [2.],
        [3.]]), batch_sizes=tensor([2, 1, 1]), sorted_indices=None, unsorted_indices=None)


In [37]:
rnn = nn.RNN(1, hidden_size, n_layers, batch_first = True)
h0 = Variable(torch.randn(n_layers, batch_size, hidden_size))
out,_ = rnn(pack,h0)

print(pack)

PackedSequence(data=tensor([[1.],
        [1.],
        [2.],
        [3.]]), batch_sizes=tensor([2, 1, 1]), sorted_indices=None, unsorted_indices=None)
torch.Size([1, 2, 2])
PackedSequence(data=tensor([[ 0.3417,  0.1451],
        [ 0.4003, -0.0267],
        [ 0.6926,  0.0129],
        [ 0.8593, -0.0580]], grad_fn=<CatBackward>), batch_sizes=tensor([2, 1, 1]), sorted_indices=None, unsorted_indices=None)


In [17]:
unpacked = nn_utils.rnn.pad_packed_sequence(out)
print(unpacked)

(tensor([[[0.3804, 0.3318],
         [0.2580, 0.8287]],

        [[0.8048, 0.7970],
         [0.0000, 0.0000]],

        [[0.9553, 0.8879],
         [0.0000, 0.0000]]], grad_fn=<CopySlices>), tensor([3, 1]))


In [66]:
#序列填充(可用)

a = (([0,1,2], [3,4], [5,6,7,8]), 1)
print(a)
# store length of each element in an array
len_a = np.array([len(a) for a in a[0]]) 
variable_a  = np.zeros((len(len_a), np.amax(len_a)))
for i, a in enumerate(a[0]):
    variable_a[i, 0:len(a)] = a
 
vocab_size = len(np.unique(variable_a))
variable_a = Variable(torch.from_numpy(variable_a))
print(variable_a)

(([0, 1, 2], [3, 4], [5, 6, 7, 8]), 1)
tensor([[0., 1., 2., 0.],
        [3., 4., 0., 0.],
        [5., 6., 7., 8.]], dtype=torch.float64)


In [ ]:
class EmbeddingLayer(nn.Module):
 
    def __init__(self, input_size, emsize):
        super(EmbeddingLayer, self).__init__()
        self.embedding = nn.Embedding(input_size, emsize)
 
    def forward(self, input_variable):
        return self.embedding(input_variable)
 
 
class Encoder(nn.Module):
 
    def __init__(self, input_size, hidden_size, bidirection):
        super(Encoder, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.bidirection = bidirection
        self.rnn = nn.LSTM(self.input_size, self.hidden_size, batch_first=True, 
                                    bidirectional=self.bidirection)
 
    def forward(self, sent_variable, sent_len):
        # Sort by length (keep idx)
        sent_len, idx_sort = np.sort(sent_len)[::-1], np.argsort(-sent_len)
        idx_unsort = np.argsort(idx_sort)
 
        idx_sort = torch.from_numpy(idx_sort)
        sent_variable = sent_variable.index_select(0, Variable(idx_sort))
 
        # Handling padding in Recurrent Networks
        sent_packed = nn.utils.rnn.pack_padded_sequence(sent_variable, sent_len, batch_first=True)
        sent_output = self.rnn(sent_packed)[0]
        sent_output = nn.utils.rnn.pad_packed_sequence(sent_output, batch_first=True)[0]
 
        # Un-sort by length
        idx_unsort = torch.from_numpy(idx_unsort)
        sent_output = sent_output.index_select(0, Variable(idx_unsort))
 
        return sent_output

In [ ]:
emb = EmbeddingLayer(vocab_size, 50)
enc = Encoder(50, 100, False, 'LSTM')
 
emb_a = emb(variable_a)
enc_a = enc(emb_a, len_a)
print(enc_a)